# Choose Model Llama 3.2 with no Fine-Tuning

In [1]:
import torch
print("CUDA Available: ", torch.cuda.is_available())
print("CUDA Device Name: ", torch.cuda.get_device_name(0))
torch.cuda.empty_cache()

# Verificar se CUDA está disponível para acelerar o processamento
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Usando dispositivo: {device}")

CUDA Available:  True
CUDA Device Name:  NVIDIA GeForce RTX 3050 Ti Laptop GPU
Usando dispositivo: cuda


## Llama 3.2 Standart

In [1]:
from unsloth import FastLanguageModel
import torch

2025-01-07 12:45:19.514439: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-07 12:45:19.524926: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-07 12:45:19.537988: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-07 12:45:19.541110: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-07 12:45:19.551474: I tensorflow/core/platform/cpu_feature_guar

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [3]:
# model_path = "../../Models/llama_3.2_FT_lora_4000_questions_short_answer_labels"
model_path = "unsloth/Llama-3.2-3B-Instruct"

# Carregar o modelo e o tokenizador separadamente
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=max_seq_length,
    dtype = dtype,
    load_in_4bit=load_in_4bit
)

==((====))==  Unsloth 2024.10.6: Fast Llama patching. Transformers = 4.46.0.
   \\   /|    GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU. Max memory: 3.712 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 8.6. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [4]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), ep

# Dataset TeleQnA for Inference

In [5]:
import json

# Path to the TeleQnA processed question in JSON file
rel17_200_questions_path = r"../../Files/rel17_200_questions.json"

# Load the TeleQnA data just release 17
with open(rel17_200_questions_path, "r", encoding="utf-8") as file:
    rel17_200_questions = json.load(file)
print(len(rel17_200_questions))

200


In [26]:
rel17_200_questions[0]

{'question': 'Which NGAP procedure is used for inter-system load balancing? [3GPP Release 17]',
 'option 1': 'eNB Configuration Transfer',
 'option 2': 'Downlink RAN Configuration Transfer',
 'option 3': 'Uplink RAN Configuration Transfer',
 'option 4': 'MME Configuration Transfer',
 'answer': 'option 3: Uplink RAN Configuration Transfer',
 'explanation': 'The NGAP procedure used for inter-system load balancing is Uplink RAN Configuration Transfer.',
 'category': 'Standards overview'}

# Import RAG Functions

In [10]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

In [11]:
from utils.rag_functions import load_faiss_index, search_faiss_index, search_RAG, load_chunks

In [12]:
# Test RAG
query_text = "reception of a transparent l3 message in unacknowledged mode"
index_file_path = "../../Files/faiss_index.bin"
chunks_path = "../../Files/tspec_chunks_markdown.pkl"
top_k = 5

In [ ]:
result = search_RAG(query_text, index_file_path, chunks_path, top_k)
print(result)


# Accuracy evaluation

## Create prompt and Ask function for Llama 3.2 Standart

In [6]:
import torch
from unsloth.chat_templates import get_chat_template

def ask_llama_3_2_No_RAG(model, tokenizer, question_data, top_k=5, index_file_path="../../Files/faiss_index.bin", chunks_path="../../Files/tspec_chunks_markdown.pkl"):
    """
    Function to generate an answer using the model based on the given question and options, 
    including relevant information from a RAG search and prompting a chain of thought.
    
    Parameters:
    - model: The language model loaded for inference.
    - tokenizer: The tokenizer configured with `get_chat_template`.
    - question_data: Dictionary containing the question and options.
    - top_k: Number of relevant chunks to retrieve from the search.
    - index_file_path: Path to the FAISS index file.
    - chunks_path: Path to the chunks file.

    Returns:
    - String: Model's generated response.
    """

    # Extract question and options
    question = question_data['question']
    options = [f"{key}: {value}" for key, value in question_data.items() if 'option' in key]
    
    # question_search = (
    #     f"{question}\n" +
    #     " ".join(options) + " "
    # )

    # # Perform RAG search using the question to retrieve relevant information
    # rag_results = search_RAG(question_search, index_file_path=index_file_path, chunks_path=chunks_path, top_k=top_k)

    # Create the prompt with Chain of Thought (CoT) instructions
    prompt = (
        # f"Relevant Information:\n{rag_results}\n"
        f"Question: {question}\n"
        f"Options:\n" + "\n".join(options) + "\n"
        # "Think step by step and analyze the relevant information carefully, then choose the correct option.\n"
        "Think step by step and choose the correct option.\n"
        "You must respond in the format 'correct option: <X>', where <X> is the correct letter for the option."
    )

    # Create the input for the model
    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    # Generate the response
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=2048,
        temperature=0.7,
        min_p=0.9,
        use_cache=True
    )

    # Clear memory
    del inputs
    torch.cuda.empty_cache()

    # Decode and return the model's output
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return response


In [8]:
# Example usage
model = FastLanguageModel.for_inference(model)  # Enable faster inference
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

question_data = {
    'question': 'Which physical channel informs the UE and the RN about the number of OFDM symbols used for the PDCCHs? [3GPP Release 17]',
    'option 1': 'PBCH',
    'option 2': 'PCFICH',
    'option 3': 'PDSCH',
    'option 4': 'PHICH',
    'answer': 'option 2: PCFICH',
    'explanation': 'The physical control format indicator channel (PCFICH) informs the UE and the RN about the number of OFDM symbols used for the PDCCHs.',
    'category': 'Standards specifications'
}

llama_3_2_response = ask_llama_3_2_No_RAG(model, tokenizer, question_data)
print(llama_3_2_response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

Question: Which physical channel informs the UE and the RN about the number of OFDM symbols used for the PDCCHs? [3GPP Release 17]
Options:
option 1: PBCH
option 2: PCFICH
option 3: PDSCH
option 4: PHICH
Think step by step and choose the correct option.
You must respond in the format 'correct option: <X>', where <X> is the correct letter for the option.assistant

To determine the correct option, let's analyze each choice in relation to the question about the physical channel that informs the UE and the RN about the number of OFDM symbols used for the PDCCHs in 3GPP Release 17.

- Option 1: PBCH (Pilot Body Channel) - This channel is used to transmit the physical channel information, including the number of OFDM symbols used for PDCCHs. It is a crucial channel for the UE to decode the PDCCHs correctly.

- Option 2: PCFICH (Pilot Channel Format Indicator Channel) - This channel is used to indicate the format of

## Evaluate Question 

In [9]:
import re

def extract_option(answer):
    """
    Extract the option part from the answer string, removing all punctuation and converting to lowercase.
    
    Parameters:
    - answer: A string containing the answer in the format 'option X: ...'.

    Returns:
    - String: Extracted option (e.g., 'option 2'), or None if no match is found.
    """
    # Remove all punctuation and convert to lowercase
    cleaned_answer = re.sub(r'[^\w\s]', '', answer.lower())
    # Find all matches for the format "option X"
    matches = re.findall(r'option \d+', cleaned_answer)
    # Return the last match with stripped whitespace if any found, otherwise None
    return matches[-1].strip() if matches else None

In [10]:
def extract_response_after_assistant(response):
    """
    Extract the part of the response that comes after the 'assistant' marker.

    Parameters:
    - response: The complete response from the model.

    Returns:
    - String: The extracted relevant part of the response.
    """
    # Split the response based on the 'assistant' marker
    parts = response.split('assistant', 1)
    # Return the part after 'assistant' or the entire response if 'assistant' is not found
    return parts[1].strip() if len(parts) > 1 else response.strip()

In [11]:
def evaluate_model_response(model_response, question_data):
    """
    Compare the model's response with the correct answer from the question data.
    
    Parameters:
    - model_response: The response string generated by the model.
    - question_data: Dictionary containing the question, options, and the correct answer.

    Returns:
    - 1 if the response is correct, otherwise the extracted model option.
    """
    correct_option = extract_option(question_data['answer'])  # Extract correct option
    relevant_response = extract_response_after_assistant(model_response)  # Get relevant part of response
    model_option = extract_option(relevant_response)  # Extract model's option

    return 1 if model_option == correct_option else model_option  # Return 1 if correct, else model's option


In [12]:
question_data = {
    'question': 'Which physical channel informs the UE and the RN about the number of OFDM symbols used for the PDCCHs? [3GPP Release 17]',
    'option 1': 'PBCH',
    'option 2': 'PCFICH',
    'option 3': 'PDSCH',
    'option 4': 'PHICH',
    'answer': 'option 2: PCFICH',
    'explanation': 'The physical control format indicator channel (PCFICH) informs the UE and the RN about the number of OFDM symbols used for the PDCCHs.',
    'category': 'Standards specifications'
}

In [ ]:
evaluation_result = evaluate_model_response(llama_3_2_response, question_data)
print(evaluation_result)

## Ask to model Llama 3.2 Standart TeleQnA questions 

### Release 17 200 questions

In [13]:
def evaluate_questions(model, tokenizer, questions):
    """
    Process all questions and return the model responses.
    
    Parameters:
    - model: The language model loaded for inference.
    - tokenizer: The tokenizer configured with `get_chat_template`.
    - questions: List of dictionaries containing question data, where each dictionary has:
        - 'question': A string representing the question to be asked to the model.
        - 'answer': A string representing the correct answer format (e.g., 'option 2: PCFICH').
        - 'response': A string that will contain the model's generated response to the question.
    
    Returns:
    - List: A list of dictionaries where each dictionary contains:
        - 'question': The question as a string.
        - 'answer': The correct answer as a string.
        - 'response': The model's generated response for that question.
    """
    
    responses = []
    total_questions = len(questions)
    
    for idx, question_data in enumerate(questions):
        response = ask_llama_3_2_No_RAG(model, tokenizer, question_data)
        responses.append({
            "question": question_data['question'],
            "answer": question_data['answer'],
            "response": response
        })
        
        # Print progress
        print(f"Responded {idx + 1} of {total_questions} questions...")

    return responses

In [14]:
model = FastLanguageModel.for_inference(model)  # Enable faster inference
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

# Process all questions and get responses
responses_llama_3_2 = evaluate_questions(model, tokenizer, rel17_200_questions)

Responded 1 of 200 questions...
Responded 2 of 200 questions...
Responded 3 of 200 questions...
Responded 4 of 200 questions...
Responded 5 of 200 questions...
Responded 6 of 200 questions...
Responded 7 of 200 questions...
Responded 8 of 200 questions...
Responded 9 of 200 questions...
Responded 10 of 200 questions...
Responded 11 of 200 questions...
Responded 12 of 200 questions...
Responded 13 of 200 questions...
Responded 14 of 200 questions...
Responded 15 of 200 questions...
Responded 16 of 200 questions...
Responded 17 of 200 questions...
Responded 18 of 200 questions...
Responded 19 of 200 questions...
Responded 20 of 200 questions...
Responded 21 of 200 questions...
Responded 22 of 200 questions...
Responded 23 of 200 questions...
Responded 24 of 200 questions...
Responded 25 of 200 questions...
Responded 26 of 200 questions...
Responded 27 of 200 questions...
Responded 28 of 200 questions...
Responded 29 of 200 questions...
Responded 30 of 200 questions...
Responded 31 of 200

In [15]:
print(responses_llama_3_2[1]['response'])

system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

Question: What is covered by enhanced application layer support for V2X services? [3GPP Release 17]
Options:
option 1: PC5 radio resource control
option 2: Advanced V2X services
option 3: SDAP layer enhancements
option 4: V2X communication over NR PC5 reference point
option 5: Tele-Operated Driving
Think step by step and choose the correct option.
You must respond in the format 'correct option: <X>', where <X> is the correct letter for the option.assistant

correct option: option 4


#### Save accuracy responses

In [16]:
def save_responses_to_json(responses, filename):
    """
    Save the model responses to a JSON file.
    
    Parameters:
    - responses: List of responses to save.
    - filename: Name of the JSON file.
    """
    
    with open(filename, "w") as json_file:
        json.dump(responses, json_file, indent=4)

In [17]:
save_responses_to_json(responses_llama_3_2,"../../Models_responses/Accuracy_larger_test/no_rag_llama_3_2_release_17_200_questions.json")

#### Evaluate responses from Llama 3.2

In [20]:
import json

# Load responses from the JSON file
with open("../../Models_responses/Accuracy_larger_test/no_rag_llama_3_2_release_17_200_questions.json", "r") as file:
    responses_llama_3_2 = json.load(file)

# Print the loaded responses to verify
print("Responses loaded")
# for response in responses_llama_3_2:
#     print(response)


Responses loaded


In [18]:
def evaluate_accuracy(responses_llama_3_2, rel_questions):
    """
    Evaluate the model's responses and calculate accuracy.
    """
    correct_count = 0  # Track the number of correct responses
    none_count = 0  # Track the number of 'None' responses

    for index, question_data in enumerate(responses_llama_3_2):
        evaluation_result = evaluate_model_response(question_data['response'], question_data)
        options = [f"{key}: {value}" for key, value in rel_questions[index].items() if 'option' in key]

        if evaluation_result == 1:
            correct_count += 1  # Increment for correct response
        elif evaluation_result is None:
            # Print only responses that are None
            print("\nWrong Answer")
            print(f"Question {index + 1}: {question_data['question']}")
            print(f"Options:\n" + "\n".join(options) + "\n")
            print(f"Full model response:\n{question_data['response']}")
            print(f"Correct response: {question_data['answer']}")
            print("----------------------------------------------------------------------------------------")
            none_count += 1  # Increment for None response
        else:
            print("\nWrong Answer")
            print(f"Question {index + 1}: {question_data['question']}")
            print(f"Options:\n" + "\n".join(options) + "\n")
            print(f"Model response: {evaluation_result}")
            print(f"Correct response: {question_data['answer']}")
            print("----------------------------------------------------------------------------------------")

    # Calculate and print accuracy
    accuracy = correct_count / len(responses_llama_3_2) * 100
    print(f"\nAccuracy: {accuracy:.2f}%")
    print(f"Total 'None' responses: {none_count}")
    print(f"'None' responses means that the model did not give an option")


In [21]:
evaluate_accuracy(responses_llama_3_2, rel17_200_questions)


Wrong Answer
Question 1: Which NGAP procedure is used for inter-system load balancing? [3GPP Release 17]
Options:
option 1: eNB Configuration Transfer
option 2: Downlink RAN Configuration Transfer
option 3: Uplink RAN Configuration Transfer
option 4: MME Configuration Transfer

Model response: option 2
Correct response: option 3: Uplink RAN Configuration Transfer
----------------------------------------------------------------------------------------

Wrong Answer
Question 2: What is covered by enhanced application layer support for V2X services? [3GPP Release 17]
Options:
option 1: PC5 radio resource control
option 2: Advanced V2X services
option 3: SDAP layer enhancements
option 4: V2X communication over NR PC5 reference point
option 5: Tele-Operated Driving

Model response: option 4
Correct response: option 2: Advanced V2X services
----------------------------------------------------------------------------------------

Wrong Answer
Question 3: What does the Load-Balancing steering

### Release 18 200 questions

In [22]:
import json

# Path to the TeleQnA processed question in JSON file
rel18_questions_path = r"../../Files/rel18_questions.json"

# Load the TeleQnA data just release 17
with open(rel18_questions_path, "r", encoding="utf-8") as file:
    rel18_questions = json.load(file)
print(len(rel18_questions))

780


In [23]:
rel18_test_size = 200
rel18_questions = rel18_questions[:rel18_test_size]
print(len(rel18_questions))

200


In [24]:
model = FastLanguageModel.for_inference(model)  # Enable faster inference
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

# Process all questions and get responses
responses_llama_3_2 = evaluate_questions(model, tokenizer, rel18_questions)

Responded 1 of 200 questions...
Responded 2 of 200 questions...
Responded 3 of 200 questions...
Responded 4 of 200 questions...
Responded 5 of 200 questions...
Responded 6 of 200 questions...
Responded 7 of 200 questions...
Responded 8 of 200 questions...
Responded 9 of 200 questions...
Responded 10 of 200 questions...
Responded 11 of 200 questions...
Responded 12 of 200 questions...
Responded 13 of 200 questions...
Responded 14 of 200 questions...
Responded 15 of 200 questions...
Responded 16 of 200 questions...
Responded 17 of 200 questions...
Responded 18 of 200 questions...
Responded 19 of 200 questions...
Responded 20 of 200 questions...
Responded 21 of 200 questions...
Responded 22 of 200 questions...
Responded 23 of 200 questions...
Responded 24 of 200 questions...
Responded 25 of 200 questions...
Responded 26 of 200 questions...
Responded 27 of 200 questions...
Responded 28 of 200 questions...
Responded 29 of 200 questions...
Responded 30 of 200 questions...
Responded 31 of 200

#### Save accuracy responses

In [25]:
save_responses_to_json(responses_llama_3_2,"../../Models_responses/Accuracy_larger_test/no_rag_llama_3_2_release_18_200_questions.json")

#### Evaluate responses from Llama 3.2 Standart

In [27]:
import json

# Load responses from the JSON file
with open("../../Models_responses/Accuracy_larger_test/no_rag_llama_3_2_release_18_200_questions.json", "r") as file:
    responses_llama_3_2 = json.load(file)

# Print the loaded responses to verify
print("Responses loaded")
# for response in responses_llama_3_2:
#     print(response)


Responses loaded


In [28]:
evaluate_accuracy(responses_llama_3_2, rel18_questions)


Wrong Answer
Question 7: When does the AMF enable or disable the 5G access stratum time distribution for a UE? [3GPP Release 18]
Options:
option 1: When UE is in CM-REJECTED state
option 2: When UE is in CM-IDLE state
option 3: When UE is in CM-REGISTERED state
option 4: When UE is in CM-DISCONNECTED state
option 5: When UE is in CM-CONNECTED state

Model response: option 3
Correct response: option 5: When UE is in CM-CONNECTED state
----------------------------------------------------------------------------------------

Wrong Answer
Question 10: What is the purpose of packet buffering in Edge Relocation? [3GPP Release 18]
Options:
option 1: To prevent packet loss during EAS migration
option 2: To optimize user plane latency
option 3: To enable simultaneous connectivity over source and target PSA
option 4: To eliminate the need for EAS IP replacement

Model response: option 2
Correct response: option 1: To prevent packet loss during EAS migration
-------------------------------------

### Other Releases 200 questions

In [29]:
import json

# Path to the TeleQnA processed question in JSON file
other_rel_questions_path = r"../../Files/other_rel_questions.json"

# Load the TeleQnA data just release 17
with open(other_rel_questions_path, "r", encoding="utf-8") as file:
    other_rel_questions = json.load(file)
print(len(other_rel_questions))

4987


In [30]:
other_rel_test_size = 200
other_rel_questions = other_rel_questions[:other_rel_test_size]
print(len(other_rel_questions))

200


In [31]:
model = FastLanguageModel.for_inference(model)  # Enable faster inference
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

# Process all questions and get responses
responses_llama_3_2 = evaluate_questions(model, tokenizer, other_rel_questions)

Responded 1 of 200 questions...
Responded 2 of 200 questions...
Responded 3 of 200 questions...
Responded 4 of 200 questions...
Responded 5 of 200 questions...
Responded 6 of 200 questions...
Responded 7 of 200 questions...
Responded 8 of 200 questions...
Responded 9 of 200 questions...
Responded 10 of 200 questions...
Responded 11 of 200 questions...
Responded 12 of 200 questions...
Responded 13 of 200 questions...
Responded 14 of 200 questions...
Responded 15 of 200 questions...
Responded 16 of 200 questions...
Responded 17 of 200 questions...
Responded 18 of 200 questions...
Responded 19 of 200 questions...
Responded 20 of 200 questions...
Responded 21 of 200 questions...
Responded 22 of 200 questions...
Responded 23 of 200 questions...
Responded 24 of 200 questions...
Responded 25 of 200 questions...
Responded 26 of 200 questions...
Responded 27 of 200 questions...
Responded 28 of 200 questions...
Responded 29 of 200 questions...
Responded 30 of 200 questions...
Responded 31 of 200

#### Save accuracy responses

In [32]:
save_responses_to_json(responses_llama_3_2,"../../Models_responses/Accuracy_larger_test/no_rag_llama_3_2_other_rel_200_questions.json")

#### Evaluate responses from Llama 3.2 Standart

In [34]:
import json

# Load responses from the JSON file
with open("../../Models_responses/Accuracy_larger_test/no_rag_llama_3_2_other_rel_200_questions.json", "r") as file:
    responses_llama_3_2 = json.load(file)

# Print the loaded responses to verify
print("Responses loaded")
# for response in responses_llama_3_2:
#     print(response)


Responses loaded


In [35]:
evaluate_accuracy(responses_llama_3_2, other_rel_questions)


Wrong Answer
Question 2: What is the empirical probability mass function denoted by ˆpn?
Options:
option 1: The proportion of times thatX=n in the firstn samples
option 2: The proportion of times thatX=n in the firstm samples
option 3: The proportion of times thatX=n in the firstn/2 samples
option 4: The proportion of times thatX=n in the firstni.i.d. samples
option 5: The proportion of times thatX=n in the lastn samples

Model response: option 4
Correct response: option 1: The proportion of times thatX=n in the firstn samples
----------------------------------------------------------------------------------------

Wrong Answer
Question 5: What has motivated an increased focus on robotic communication systems?
Options:
option 1: Advancements in manufacturing processes
option 2: Increasing difficulty and complexity of robotic tasks
option 3: Gathering and processing of high dimensional data
option 4: Emerging wireless networks with high data rates and stability
option 5: All of the abo